In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import os

if not os.path.isdir('/content/drive/MyDrive'):
    drive.mount('/content/drive')

OUTPUT_DIR = '/content/drive/MyDrive/Careerwill_Courses'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Output:', OUTPUT_DIR)

In [ ]:
# Step 2: Install system deps
%%bash
apt-get update -qq
apt-get install -y -qq ffmpeg aria2 > /dev/null 2>&1
wget -q https://github.com/shaka-project/shaka-packager/releases/download/v2.6.1/packager-linux-x64 -O /usr/local/bin/shaka-packager 2>/dev/null
chmod +x /usr/local/bin/shaka-packager
pip install -q yt-dlp requests beautifulsoup4 lxml

echo 'Dependencies installed.'

In [ ]:
# Step 3: Clone repo (if not present) and install package deps
import os
REPO_DIR = '/content/careerwill-downloader'
if not os.path.exists(REPO_DIR):
    !git clone https://github.com/PerspicaciousGuy/Advance-Udemy-Downloader.git {REPO_DIR}

os.chdir(REPO_DIR)
!pip install -q requests beautifulsoup4 lxml yt-dlp
print('Repository ready at', REPO_DIR)

In [ ]:
# Step 4: Configuration - Paste your auth below
# Use either BEARER_TOKEN (recommended) or COOKIES (Netscape format). Set USE_BEARER_TOKEN accordingly.
USE_BEARER_TOKEN = True
BEARER_TOKEN = ""  # Paste bearer token here

COOKIES = """
# Paste Netscape format cookies here (optional)
"""

# Widevine keys (optional - one per line, KID:KEY)
WIDEVINE_KEYS = """
"""

# Course URL to test (paste a Careerwill course or lesson URL)
COURSE_URL = "https://web.careerwill.com/your-course-url"

# Parse widevine keys into dict
keys = {}
for line in WIDEVINE_KEYS.strip().split('\n'):
    if ':' in line and line.strip():
        kid, key = line.strip().split(':', 1)
        keys[kid.strip()] = key.strip()

# Save cookies to a file if provided
if not USE_BEARER_TOKEN and COOKIES.strip():
    with open('cookies.txt','w',encoding='utf8') as f:
        f.write(COOKIES.strip())

print('Config ready')

In [ ]:
# Step 5: Run the Careerwill downloader scaffold
from careerwill import CareerwillDownloader

if USE_BEARER_TOKEN and not BEARER_TOKEN.strip():
    raise SystemExit('Set BEARER_TOKEN or set USE_BEARER_TOKEN = False')

down = CareerwillDownloader(bearer_token=(BEARER_TOKEN.strip() if USE_BEARER_TOKEN else None),
                             cookies_file=('cookies.txt' if not USE_BEARER_TOKEN else None),
                             widevine_keys=keys,
                             output_dir=OUTPUT_DIR)

print('Starting discovery...')
down.download_course(COURSE_URL)
print('Done')